In [1]:
import numpy as np

In [2]:
from pybhpt.flux import FluxMode
from pybhpt.geo import KerrGeodesic
from pybhpt.teuk import TeukolskyMode
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
def convergenceTestModes(fluxMode, fluxTot, fluxComp, fluxComp2, tol = 1e-8, flux_position=[0, 1, 2]):
    if np.all(np.abs(np.array(fluxMode))[flux_position]/np.array(fluxTot)[flux_position] < tol) and np.all(np.abs(np.array(fluxComp))[flux_position]/np.array(fluxTot)[flux_position] < tol) and np.all(np.abs(np.array(fluxMode)[flux_position]) < np.abs(np.array(fluxComp)[flux_position])) and np.all(np.abs(np.array(fluxMode)[flux_position]) < np.abs(np.array(fluxComp2)[flux_position])):
        return True
    else:
        return False

class FluxSummation:
    '''
    Class for summing over all of the frequency modes that contribute to the total gravitational wave (GW()) fluxes
    produced by a particle on a geodesic orbit parametrized by the parameters $(a, p, e, x)$

    :param a: Kerr spin parameter
    :param p: Semi-latus rectum
    :param e: Orbital eccentricity
    :param x: Projection of the orbital inclination
    :param nsamples0: Initial number of sampling points to use in orbital integration of fluxes

    :attribute totalfluxes: Three dimensional array of GW fluxes, [dE/dt, dL/dt, dQ/dt]
    :attribute horizonfluxes: Three dimensional array of GW fluxes radiated down the black hole horizon
    :attribute infinityfluxes: Three dimensional array of GW fluxes radiated away at infinity
    '''
    def __init__(self, a, p, e, x, nsamples0 = None, tol = 1.e-8):
        self.tol = tol
        self.a = a
        self.p = p
        self.e = e
        self.x = x
        if nsamples0 is None:
            nsamples0 = 2**7
        self.nsamples0 = nsamples0
        self.nsamples = nsamples0
        self.geo_dict = {self.nsamples: KerrGeodesic(self.a, self.p, self.e, self.x, self.nsamples) }
        self.geo = self.geo_dict[self.nsamples]
        self.nlimit = 300
        self.klimit = 100
        self.mlimit = 100
        self.llimit = 100
        self.fluxref = 0.5*np.array([32./5.*np.abs(self.geo.frequencies[-1])**(10./3.), 32./5.*np.abs(self.geo.frequencies[-1])**(7./3.), 64./5.*np.abs(self.geo.frequencies[-1])**(2.)])
        print(f"Flux reference: {self.fluxref}")

        # self.nfluxes_full = np.zeros((2*self.nlimit, 3))
        # self.kfluxes_full = np.zeros((2*self.klimit, 3))
        self.totalfluxes = np.zeros(3)
        self.horizonfluxes = np.zeros(3)
        self.infinityfluxes = np.zeros(3)
        self.nfluxes = np.zeros(3)
        self.kfluxes = np.zeros(3)
        self.mfluxes = np.zeros(3)
        self.nmax = 0
        self.nmin = 0
        self.kmax = 0
        self.kmin = 0
        self.kmaxmodenums = np.zeros((2*self.nlimit + 1, 2), dtype=int)
        self.mmaxmodenums = np.zeros((2*self.nlimit + 1, 2*self.klimit + 1, 2), dtype=int)
        self.lmaxmodenums = np.zeros((2*self.nlimit + 1, 2*self.klimit + 1, self.mlimit + 1, 2), dtype=int)
        
        self.kpeakmodenums = np.zeros((2*self.nlimit + 1), dtype=int)
        self.kpeakmodes = np.zeros((2*self.llimit + 1, 3))
        self.mpeakmodenums = np.zeros((2*self.klimit + 1), dtype=int)
        self.mpeakmodes = np.zeros((2*self.llimit + 1, 3))
        self.fluxamplitudes = np.zeros((self.llimit + 1, self.mlimit + 1, 2*self.klimit + 1, 2*self.nlimit + 1), dtype=np.complex128)

        self.n = 0
        self.k = 0
        self.m = 0
        self.l = 2
        self.flux_positions = [0, 1, 2]

    def check_for_peak_m_mode(self):
        current_peak_value = self.mpeakmodes[self.k]
        if np.all(np.abs(self.mfluxes) > current_peak_value):
            current_peak_value = np.abs(self.mfluxes)
            self.mpeakmodes[self.k] = current_peak_value
            self.mpeakmodenums[self.k] = self.m
            # print(f"New m-peak is {self.m} for {(self.k, self.n)} with flux values {self.mfluxes}")

    def check_for_peak_k_mode(self):
        current_peak_value = self.kpeakmodes[self.n]
        if np.all(np.abs(self.kfluxes) > current_peak_value):
            current_peak_value = np.abs(self.kfluxes)
            self.kpeakmodes[self.n] = current_peak_value
            self.kpeakmodenums[self.n] = self.k
            # print(f"New k-peak is {self.k} for {(self.n)} with flux values {self.kfluxes}")

    '''
    Sums over all possible frequency mode contributions, starting with the radial $n$-mode frequencies, then azimuthal $m$-modes, then spheroidal $l$-modes
    '''
    def sum(self, nmin_include = 3, kmin_include = 3, print_output_n_sum = False, print_output_k_sum = False, print_output_m_sum = False, tol = None):
        if tol is not None:
            self.tol = tol
        self.total_mode_summation(nmin_include = nmin_include, kmin_include = kmin_include, print_output_k_sum = print_output_k_sum, print_output_n_sum = print_output_n_sum, print_output_m_sum = print_output_m_sum)

    def total_mode_summation(self, nmin_include = 3, kmin_include = 3, print_output_n_sum = False, print_output_k_sum = False, print_output_m_sum = False):
        # sum over positive m-modes
        self.n_mode_summation(0, nmin_include = nmin_include, kmin_include = kmin_include, increment = 1, print_output_n_sum = print_output_n_sum, print_output_k_sum = print_output_k_sum, print_output_m_sum = print_output_m_sum)
        # self.mmaxmodenums[self.n][1] = self.n - 1

        # sum over negative m-modes
        self.n_mode_summation(-1, nmin_include = nmin_include, kmin_include = kmin_include, increment = -1, print_output_n_sum = print_output_n_sum, print_output_k_sum = print_output_k_sum, print_output_m_sum = print_output_m_sum)
        # self.mmaxmodenums[self.n][0] = self.n + 1

    def increase_geo_sampling(self, nsamples):
        self.nsamples = nsamples
        if self.nsamples in self.geo_dict.keys():
            self.geo = self.geo_dict[self.nsamples]
        else:
            self.geo_dict[self.nsamples] = KerrGeodesic(self.a, self.p, self.e, self.x, nsamples = self.nsamples)
            self.geo = self.geo_dict[self.nsamples]


    def n_mode_summation(self, ninit, nmin_include = 3, kmin_include = 3, increment = 1, print_output_n_sum = False, print_output_k_sum = False, print_output_m_sum = False):
        convergedN = False

        # set n based on initialied value
        self.n = ninit

        kpeak = abs(ninit)
        previous_nfluxes = np.zeros(3)
        previous_nfluxes_2 = np.zeros(3)

        while (abs(self.n) < self.nlimit and convergedN is False) or (abs(nmin_include) < 3):
            if abs(8*self.n) > self.nsamples:
                self.increase_geo_sampling(2*self.nsamples)
            if abs(self.n) > 84 and self.nsamples < 2**11:
                self.increase_geo_sampling(2**11)

            self.nfluxes = np.zeros(3)
            self.k_mode_summation(kpeak, kmin_include = kmin_include, increment = 1, print_output_k_sum = print_output_k_sum, print_output_m_sum = print_output_m_sum)
            kmin = 1
            self.kmaxmodenums[self.n] = [kmin, self.k - 1] # record the max k-modes used for the sum
            self.k_mode_summation(kpeak - 1, kmin_include = kmin_include, increment = -1, print_output_k_sum = print_output_k_sum, print_output_m_sum = print_output_m_sum)
            if self.k + 1 < kmin:
                self.kmaxmodenums[self.n][0] = self.k + 1

            convergedN = convergenceTestModes(self.nfluxes, self.fluxref, previous_nfluxes, previous_nfluxes_2, flux_position=self.flux_positions, tol=self.tol)
            previous_nfluxes_2 = previous_nfluxes
            previous_nfluxes = self.nfluxes
            # self.nfluxes_full[self.n] = self.nfluxes
            if print_output_n_sum:
                print("n summation results (kmin, kmax, n)")
                print(self.kmaxmodenums[self.n][0], self.kmaxmodenums[self.n][1], self.n, self.nfluxes)
            
            kpeak = self.kpeakmodenums[self.n]
            self.n += increment

    def k_mode_summation(self, kinit, kmin_include = 3, increment = 1, print_output_k_sum = False, print_output_m_sum = False):
        include_zero = True
        # if increment > 0: # don't include negative m-modes to avoid double counting
        #     include_zero = False
        
        convergedK = False
        self.k = kinit
        mpeak = abs(kinit + 1)
        if mpeak <= 0:
            mpeak = 1
        previous_kfluxes = np.zeros(3)
        previous_kfluxes_2 = np.zeros(3)
        convergedCount = 0

        # while (abs(self.k) < self.klimit and convergedK is False) or (abs(kmin_include) < 3):
        while (abs(self.k) < self.klimit and convergedCount < 3) or (abs(kmin_include) < 3):

            if abs(32*self.k) > self.nsamples:
                self.increase_geo_sampling(2*self.nsamples)
            if abs(self.k) > 84 and self.nsamples < 2**11:
                self.increase_geo_sampling(2**11)

            # if the polar and radial contributions to the frequency are positive, include m = 0
            # if they are negative, do not include m = 0 to avoid double counting
            if self.k*self.geo.frequencies[1] + self.n*self.geo.frequencies[0] > 0:
                include_zero = True
            else:
                include_zero = False

            # perform sum with increasing m, starting at mpeak
            self.kfluxes = np.zeros(3)
            self.m_mode_summation(mpeak, increment = 1, include_zero = include_zero, print_output = print_output_m_sum)
            
            # set minimum m value based on whether we include m = 0 for this value of (k,n)
            mmin = 1
            if include_zero:
                mmin = 0
            self.mmaxmodenums[self.k] = [mmin, self.m - 1] # record the max m-modes used for the sum
            
            # perform sum with decreasing m, starting at mpeak - 1 and stopping either
            # when the sum converges or m = mmin
            if mpeak > 0:
                self.m_mode_summation(mpeak - 1, increment = -1, include_zero = include_zero, print_output = print_output_m_sum)

            # check to see where the k-modes are peaking as we sum over k
            self.check_for_peak_k_mode()
            if self.m + 1 < mmin:
                self.mmaxmodenums[self.k][0] = self.m + 1

            # add the latest k-mode to the n-mode sum
            self.nfluxes += self.kfluxes

            # check if the k-modes are passing the convergence tests
            convergedK = convergenceTestModes(self.kfluxes, self.fluxref, previous_kfluxes, previous_kfluxes_2, flux_position=self.flux_positions, tol=self.tol)
            if convergedK is True:
                convergedCount += 1

            # store most recent k-mode flux values
            previous_kfluxes_2 = previous_kfluxes
            previous_kfluxes = self.kfluxes

            if print_output_k_sum:
                print("k summation results (mmax, k, n)")
                print(self.mmaxmodenums[self.k][1], self.k, self.n, self.kfluxes)
            
            mpeak = self.mpeakmodenums[self.k]
            self.k += increment

    def m_mode_summation(self, minit, increment = 1, include_zero = False, print_output = False):
        # increment sets how much to change m. Usually set to +1 or -1
        mmin = 1
        if include_zero:
            mmin = 0
        convergedM = False
        self.m = minit
        previous_mfluxes = np.zeros(3)
        previous_mfluxes_2 = np.zeros(3)
        while (self.m < self.mlimit and self.m >= mmin and convergedM is False):
            # increase sampling rate for high m-modes
            if 8*self.m > self.nsamples:
                self.increase_geo_sampling(2*self.nsamples)
            # perform sum over l-modes
            if self.m == 0:
                self.flux_positions = 0 # we ignore angular momentum for m = 0 because it vanishes
            else:
                self.flux_positions = [0, 1, 2]
                
            self.mfluxes = np.zeros(3)
            self.l_mode_summation()
            self.check_for_peak_m_mode()

            # add this m-mode contribution to n-mode and test for convergence
            self.kfluxes += self.mfluxes
            convergedM = convergenceTestModes(self.mfluxes, self.fluxref, previous_mfluxes, previous_mfluxes_2, flux_position=self.flux_positions, tol=self.tol)
            previous_mfluxes_2 = previous_mfluxes
            previous_mfluxes = self.mfluxes

            if print_output:
                print("m summation results")
                print(self.l - 1, self.m, self.k, self.n, self.mfluxes)
            self.m += increment

        if self.m < -1:
            print(self.m, self.k, self.n, mmin, include_zero)
        self.flux_positions = [0, 1, 2]

    def l_mode_summation(self, precision_tolerance = None, store_amplitudes = False):
        # initialize convergence criteria to False
        if precision_tolerance is None:
            precision_tolerance = self.tol

        convergedL = False
        previous_lfluxes = np.zeros(3)
        error_ref = precision_tolerance*self.fluxref

        # find minimum l-mode
        lmin = np.max([self.m, 2])
        self.l = lmin

        nsamples = self.nsamples
        # sum over l-modes until we hit the l-limit or until the convergence test returns True
        sample_increase_flag = 0
        convergeCount = 0
        # while self.l <= self.llimit and convergedL is False:
        while self.l <= self.llimit and convergeCount < 2:
            # calculate Teukolsky mode
            teuk = TeukolskyMode(-2, self.l, self.m, self.k, self.n, self.geo)
            flux = FluxMode(self.geo, teuk) # extract fluxes from the Teukolsky mode

            precision_improvement = 1.1
            fluxError = np.sqrt((2.*np.array(flux.horizonfluxes)*teuk.precision('In'))**2 + (2.*np.array(flux.infinityfluxes)*teuk.precision('Up'))**2)
            # if teuk.precision('Up') > precision_tolerance or teuk.precision('In') > precision_tolerance:
            if np.any(fluxError > error_ref):
                # print("In and Up values losing precision for "+str(self.l)+", "+str(self.m)+", "+str(self.n))
                # print(f"Flux error is {fluxError}")
                # while precision_improvement > 1. and sample_increase_flag < 5:
                while np.any(fluxError > error_ref) and sample_increase_flag < 5:
                    self.increase_geo_sampling(2*self.nsamples)
                    teuk_temp = TeukolskyMode(-2, self.l, self.m, self.k, self.n, self.geo)
                    flux_temp = FluxMode(self.geo, teuk_temp)
                    fluxError = np.sqrt((2.*np.array(flux_temp.horizonfluxes)*teuk_temp.precision('In'))**2 + (2.*np.array(flux_temp.infinityfluxes)*teuk_temp.precision('Up'))**2)
                    # precision_improvement_temp = np.max([teuk.precision('Up')/teuk_temp.precision('Up'), teuk.precision('In')/teuk_temp.precision('In')])
                    # precision_improvement = precision_improvement_temp
                    # print("In and Up values losing precision for "+str(self.l)+", "+str(self.m)+", "+str(self.n))
                    # print(f"Flux error is {fluxError}")
                    sample_increase_flag += 1
                    if self.l == lmin or self.l == lmin + 1:
                        nsamples = self.nsamples # if we already need to increase sampling rate at the beginning of the sum, permanently increase sampling rate
                
                # if teuk_temp.precision('Up') < precision_tolerance or teuk_temp.precision('In') < precision_tolerance:
                if np.any(fluxError < error_ref):
                    teuk = teuk_temp
                    flux = flux_temp

            # if the Teukolsky calculation did not lose too much precision, continue with calculation
            totalref = 0.*np.array(flux.totalfluxes)
            if np.any((2.*np.array(flux.infinityfluxes)*teuk.precision('Up')) < error_ref):
                self.infinityfluxes += flux.infinityfluxes
                totalref += flux.infinityfluxes
                if store_amplitudes:
                    self.fluxamplitudes[self.l, self.m, self.n] = 0.5*teuk.amplitude('Up')/teuk.frequency**2 # store waveform amplitude for later use

            if np.any((2.*np.array(flux.horizonfluxes)*teuk.precision('In')) < error_ref):
                self.horizonfluxes += flux.horizonfluxes
                totalref += flux.horizonfluxes
            
            self.mfluxes += totalref # add to m-mode test
            self.totalfluxes += totalref # add to total fluxes
            
            if np.all(self.totalfluxes > self.fluxref):
                self.fluxref = self.totalfluxes
            
            # test for convergence
            convergedL = convergenceTestModes(totalref, self.fluxref, previous_lfluxes, previous_lfluxes, flux_position=self.flux_positions, tol=self.tol)
            if convergedL is True:
                convergeCount += 1
            previous_lfluxes = totalref # store this l-mode to compare to the next l-mode
            self.l += 1
        
        self.nsamples = nsamples
        self.geo = self.geo_dict[nsamples]

        # record the min and max l-modes used for the sum
        self.lmaxmodenums[self.n, self.k, self.m] = [lmin, self.l]

In [9]:
from datetime import datetime
spin = 0.9
e0 = 0.1
p0 = 6
x0 =  np.cos(40/180*np.pi)
print(p0, e0)
print(datetime.now().strftime("%H:%M:%S"))
summer = FluxSummation(spin, p0, e0, x0, tol=1e-10)
summer.sum(print_output_n_sum = True, print_output_k_sum = True, print_output_m_sum = True)
print(datetime.now().strftime("%H:%M:%S"))
print('dE/dt = ', 2.*summer.totalfluxes[0])
print('dL/dt = ', 2.*summer.totalfluxes[1])
print('dQ/dt = ', 2.*summer.totalfluxes[2])

6 0.1
15:11:17
Flux reference: [0.00035882 0.00549491 0.02729088]
m summation results
7 1 0 0 [-7.97310493e-09 -1.22098052e-07 -1.63347770e-07]
m summation results
8 2 0 0 [0.00014164 0.00216904 0.00290183]
m summation results
9 3 0 0 [1.69899187e-05 2.60179190e-04 3.48078367e-04]
m summation results
10 4 0 0 [2.29180078e-06 3.50960401e-05 4.69529186e-05]
m summation results
10 5 0 0 [2.97788882e-07 4.56026137e-06 6.10090426e-06]
m summation results
12 6 0 0 [3.43094501e-08 5.25405982e-07 7.02909622e-07]
m summation results
12 7 0 0 [3.13083297e-09 4.79447607e-08 6.41424627e-08]
m summation results
13 8 0 0 [1.67457318e-10 2.56439775e-09 3.43075625e-09]
m summation results
12 9 0 0 [2.85699826e-13 4.37513272e-12 5.85323158e-12]
m summation results
14 10 0 0 [3.79494604e-12 5.81148152e-11 7.77483778e-11]
m summation results
15 11 0 0 [2.99842773e-12 4.59171413e-11 6.14298305e-11]
m summation results
16 12 0 0 [1.18423854e-12 1.81351205e-11 2.42619063e-11]
m summation results
16 13 0 0 [